In [1]:
import re
import json
import numpy

In [2]:
majors = ["ComputerSciencesBS", "DataScienceBS", "BotanyBS", "DataScienceCertificate", "GeneticsandGenomicsBS",
          "MathematicsBS", "MolecularandCellBiologyBS", "PhysicsBS"]

In [3]:
d = {}

for major in majors:
    with open("majors/"+major+".json") as f:
        d[major] = (json.load(f))

In [4]:
num_strings = "|".join(['one', 'two', 'three', 'four', 'five', 'six', 'seven', 'eight', 'nine', 'ten', '1', '2', '3', '4', '5', '6', '7', '8', '9', '10'])
num_strings

'one|two|three|four|five|six|seven|eight|nine|ten|1|2|3|4|5|6|7|8|9|10'

In [5]:
d_clean = {}

for major in d:
    d_clean[major] = {}
    idx = 0
    for item in d[major]:
        idx += 1
        key = list(item.keys())[0]+"*"*idx
        value = list(item.values())[0]
        if len(value) == 0:
            continue
        last_val=0
        val = []
        if isinstance(value, list):
            for v in value:
                if last_val and v.startswith("or"):
                    val[last_val-1] = val[last_val-1]+v
                else:
                    val.append(v)
                    last_val += 1
        else:
            val=value
        # if major == "DataScienceBS":
        #     print(val)
        d_clean[major][key] = val
        
d_clean["DataScienceBS"]

{'Required Courses*': ['MATH221orMATH217',
  'MATH222',
  'STAT240',
  'STAT340',
  'COMPSCI220orCOMPSCI300',
  'COMPSCI320',
  'LIS461orECE/ISYE570'],
 'Complete one of the following:***': ['COMPSCI/ECE/ME532',
  'COMPSCI/ECE/ME539',
  'COMPSCI540',
  'GENBUS656',
  'ISYE521',
  'MATH535',
  'PHYSICS361',
  'STAT451',
  'STAT453'],
 'Complete one of the following:*****': ['COMPSCI400',
  'COMPSCI412',
  'COMPSCI/STAT471',
  'COMPSCI/MATH513',
  'COMPSCI/MATH514',
  'COMPSCI/ECE/ISYE524',
  'COMPSCI544',
  'COMPSCI564',
  'COMPSCI565',
  'COMPSCI/BMI576',
  'GEOG573',
  'GEOG574',
  'MATH444'],
 'Complete one of the following:******': ['ECON400',
  'ECON410',
  'ECON460',
  'GEOG579',
  'ISYE575',
  'STAT/MATH309orSTAT311orMATH/STAT431',
  'STAT/MATH310orSTAT312',
  'STAT349',
  'STAT351',
  'STAT421',
  'STAT/ME424',
  'STAT436',
  'STAT443',
  'STAT456',
  'STAT461',
  'STAT575',
  'MATH531',
  'MATH/ISYE/OTM/STAT632',
  'MATH635'],
 'Complete one from the following. Only one course 

In [6]:
re.findall(r"(\d+)\s(unique )?credits", "2 credits")

[('2', '')]

In [7]:
def extract_course_count(line):
    # Regex pattern to find phrases like "complete X", "select X", etc.
    pattern = r"(complete|select|one|two|three|four|five|six|seven|eight|nine|ten|at least)\s*(\d+|one|two|three|four|five|six|seven|eight|nine|ten)?\s*(courses?|options?|from|of the following)?"
    CREDITS_PER_COURSE = 3
    
    credit_first = r"credit"
    credit = re.search(credit_first, line.lower())
    if credit:
        credit_pattern = r"(\b(\d)\s*(unique)?\s*credits)"
        match_credit = re.search(credit_pattern, line.lower())
        if match_credit:
            return match_credit.group(1)
        return 0
    
    match = re.search(pattern, line.lower())
    if match:
        # Extracting the numeric or word part
        word_or_number = match.group(2)

        # Convert word to number if needed
        words_to_numbers = {
            "one": 1, "two": 2, "three": 3, "four": 4, "five": 5,
            "six": 6, "seven": 7, "eight": 8, "nine": 9, "ten": 10
        }

        if word_or_number is None:
            return 1  # Default to 1 if no number is explicitly mentioned
        elif word_or_number.isdigit():
            return int(word_or_number)
        elif word_or_number in words_to_numbers:
            return words_to_numbers[word_or_number]

    return 0  # No courses found

In [8]:
d_nums = {}

for major in d_clean:
    d_nums[major] = []
    for item in d_clean[major]:
        num = extract_course_count(item)
        if item.lower().find("breadth") != -1:
            num = 2
        if item.lower().find("depth") != -1:
            num = 3
        if num == 0:
            d_nums[major].append((d_clean[major][item], len(d_clean[major][item])))
        else:
            d_nums[major].append((d_clean[major][item], num))

In [9]:
# d_loose = {}

# for major in d_nums:
#     d_loose[major] = []
#     for item in d_nums[major]:
#         g = item[0]
#         if isinstance(g, list):
#             for course in g:
#                 if isinstance(course, str):
#                     if course.find("&") != -1:
#                         l = course.split("&")
#                         d_loose[major].append((l, len(l)))
#                     elif course.find("or") != -1:
#                         course = (course.split("or"),1)
#                         d_loose[major].append(course)
                    
#         else:
#             d_loose[major].append(item)

# d_loose['GeneticsandGenomicsBS']

In [41]:
d_clean['PhysicsBS']

{'First Introductory Course (complete one):*': ['PHYSICS247',
  'PHYSICS207',
  'PHYSICS201',
  'EMA201&EMA202',
  'EMA201&ME240'],
 'Second Introductory Course (complete one):**': ['PHYSICS248',
  'PHYSICS208',
  'PHYSICS202'],
 'Third Introductory Course (complete one):***': ['PHYSICS249',
  'PHYSICS205',
  'PHYSICS/ECE235',
  'PHYSICS241',
  'PHYSICS311'],
 'Electromagnetism (complete one):****': ['PHYSICS322',
  'ECE220&ECE320&ECE420'],
 'Quantum Mechanics (complete one):*****': ['PHYSICS448',
  'PHYSICS531',
  'PHYSICS307',
  'PHYSICS407'],
 'Two credits apply from each of the following:******': ['PHYSICS321',
  'PHYSICS325',
  'PHYSICS623',
  'PHYSICS625',
  'NE427',
  'NE428'],
 'One credit applies from each of the following:*******': ['ECE305', 'ECE313'],
 'Advanced Physics Electives********': ['PHYSICS301',
  'PHYSICS307',
  'PHYSICS311',
  'PHYSICS321',
  'PHYSICS322',
  'PHYSICS323',
  'PHYSICS325',
  'PHYSICS406',
  'PHYSICS407',
  'PHYSICS415',
  'PHYSICS448',
  'PHYSICS44

In [10]:
majors
# d_nums['Geneti']

['ComputerSciencesBS',
 'DataScienceBS',
 'BotanyBS',
 'DataScienceCertificate',
 'GeneticsandGenomicsBS',
 'MathematicsBS',
 'MolecularandCellBiologyBS',
 'PhysicsBS']

In [11]:
## Compare two majors:

m1 = 'DataScienceCertificate'
m2 = 'GeneticsandGenomicsBS'

c1 = d_nums[m1]
c2 = d_nums[m2]

h1 = {}
for i in range(len(c1)):
    for item in c1[i][0]:
        h1[item] = h1.get(item, [])+[i]
        
h2 = {}
for i in range(len(c2)):
    for item in c2[i][0]:
        h2[item] = h2.get(item, [])+[i]
        
sorted1 = sorted(h1.items(), key=lambda item: (re.findall("\d{3}", item[0])[0], item[1][0]))
sorted2 = sorted(h2.items(), key=lambda item: (re.findall("\d{3}", item[0]+"999")[0], item[1][0]))

In [39]:
h1

{'COMPSCI220orCOMPSCI320': [0],
 'STAT240': [0],
 'ECE204': [0],
 'LIS461orECE/ISYE570': [1],
 'BIOCORE382': [2],
 'BIOCORE384': [2],
 'BIOCORE486': [2],
 'COMPSCI320': [2],
 'COMPSCI/ECE/ME532': [2],
 'COMPSCI544': [2],
 'COMPSCI565': [2],
 'COMPSCI/BMI576': [2],
 'ECON315': [2],
 'ECON400': [2],
 'ECON410': [2],
 'ECON460': [2],
 'ECON570': [2],
 'ECON695': [2],
 'EDPSYCH551': [2],
 'FINANCE310': [2],
 'GENBUS656': [2],
 'GEOG378': [2],
 'GEOG573': [2],
 'GEOG574': [2],
 'GEOG579': [2],
 'ISYE412': [2],
 'ISYE521': [2],
 'MATH444': [2],
 'MATH535': [2],
 'PHYSICS361': [2],
 'SOC362': [2],
 'SOILSCI585': [2],
 'STAT340': [2],
 'STAT405': [2],
 'STAT436': [2],
 'STAT/COMPSCI471': [2],
 'AAE/ECON421': [2],
 'BIOCHEM570': [2],
 'COMPSCI/ECE/ISYE524': [2],
 'GENBUS307': [2],
 'INFOSYS322': [2],
 'LIS440': [2],
 'LSC460': [2],
 'LSC660': [2],
 'SOC351': [2],
 'SOC/C&ESOC618': [2]}

In [25]:
import sequence_graph
import submatrix
from itertools import permutations

In [26]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [267]:
DNA = 'COMPSCI300|COMPSCI220|MATH217|MATH221|MATH222|STAT240|COMPSCI320|MATH320|STAT340|COMPSCI576|COMPSCI513|COMPSCI471|COMPSCI412|COMPSCI400|STAT309|ECON410|ECON400|COMPSCI533|COMPSCI525|COMPSCI425|ECE570|LIS461|COMPSCI539|MATH535|ISYE521|GENBUS656|COMPSCI540|COMPSCI539|COMPSCI532|CHEM115|CHEM116|CHEM109|CHEM103CHEM104|PHYSICS207|PHYSICS208|PHYSICS201|PHYSICS202|PHYSICS103PHYSICS104|BIOCORE381|BIOCORE383|BOTANY130|ZOOLOGY101|ZOOLOGY102|BIOLOGY151BIOLOGY152|MATH171|MATH217|MATH221|CHEM343CHEM345|CHEM341|STAT240|STAT301|STAT371|GENETICS466|GENETICS467GENETICS468|BIOCHEM507|BIOCHEM501|BIOCHEM570|COMPSCI576|BIOCHEM560|BIOCHEM508|GENETICS528|GENETICS525|GENETICS520|GENETICS527|GENETICS522|GENETICS299|GENETICS545'.split("|")
basic_dna_submatrix = submatrix.match_mismatch_matrix(1, -1, DNA)
# basic_dna_submatrix

In [268]:
def align_graphs(x, y, substitution_matrix, s):
    """Computes a global pairwise alignment of two sequence graphs
    
    Uses a linear gap scoring function with space score s.
    The graphs are assumed to be in topological order, i.e., if vertex j is a parent of vertex i, then j < i
    In the case of ties during the traceback, the algorithm will traceback to the cell (i, j) in
    the dynamic programming matrix with the largest coordinates.
    
    Args:
        x: a sequence graph (a SequenceGraph object).
        y: a sequence graph (a SequenceGraph object).
        substitution_matrix: the substitution matrix, represented as a dictionary 
        s: the space score (a numeric value)
    Returns:
        A tuple (score, alignment), where alignment is represented
        as a list two aligned strings of the same length.
    """
    assert x.is_valid(), "sequence graph x is not valid"
    assert y.is_valid(), "sequence graph y is not valid"

    ###
    ### Your solution to PROBLEM 1
    ###
    
    n = x.num_vertices()
    m = y.num_vertices()  

    M = matrix(n, m)
    t = matrix(n, m)
    
    M[0][0]=0
    
    for i in range(1, n):
        parents = [(M[k][0]+s, (k,0)) for k in sorted(x.parents(i), reverse=True)]
        M[i][0], t[i][0] = max(parents)
        
    for j in range(1, m):
        parents = [(M[0][l]+s, (0,l)) for l in sorted(y.parents(j), reverse=True)]
        M[0][j], t[0][j] = max(parents)
        
    
    for i in range(1, n-1):
        for j in range(1, m-1):
            vals1 = []
            for k in sorted(x.parents(i), reverse=True):
                for l in sorted(y.parents(j), reverse=True):
                    vals1.append((M[k][l] + substitution_matrix[x.vertex_label(i), y.vertex_label(j)], (k,l)))
                    
            val1 = max(vals1)
            vals2 = [(M[k][j]+s, (k, j)) for k in sorted(x.parents(i), reverse=True)]
            val2 = max(vals2)
            vals3 = [(M[i][l]+s, (i, l)) for l in sorted(y.parents(j), reverse=True)]
            val3 = max(vals3)
            vals = sorted([val1, val2, val3], key=lambda item:item[1], reverse=True)
            M[i][j], t[i][j] = max(vals)
    
    vals = []
    for k in x.parents(n-1):
        for l in y.parents(m-1):
            vals.append((M[k][l], (k,l)))
    M[n-1][m-1], t[n-1][m-1] = max(vals)
    
    x_result = ""
    y_result = ""
    i, j = n-1,m-1
    
    pairs = []
    while i or j:
        traceback = t[i][j]
        p, q = traceback
                    
        if p == i:
            pairs.append(("-", y.vertex_label(j)))
        
        elif q == j:
            pairs.append((x.vertex_label(i), "-"))
            
        else:
            pairs.append((x.vertex_label(i) or "-", y.vertex_label(j) or "-"))
            
        while j-q > 1:
            j -= 1
            pairs.append((" ", y.vertex_label(j)))
        
        while i-p > 1:
            i -= 1
            pairs.append((x.vertex_label(i), " "))

                
        i = p
        j = q

    pairs.reverse()
    pairs = pairs[:-1]    
    
    return (M[n-1][m-1], pairs) #transpose_alignment(pairs))
    
# Some helper functions that may be of use to you in your implementation
def matrix(num_rows, num_cols, initial_value=None):
    """Returns a matrix (a list of rows, each of which is a list) 
    with num_rows and num_cols and with initial_value in each entry"""
    return [[initial_value] * num_cols for i in range(num_rows)]

def transpose_alignment(alignment):
    """Returns a column-based alignment from a row-based alignment or vice versa"""
    return list(map(''.join, zip(*alignment)))

In [269]:
ds = "|".join("""COMPSCI220
MATH221
MATH222
STAT240
COMPSCI320
MATH320
STAT340
COMPSCI400
ECON400
COMPSCI425
LIS461
COMPSCI532""".split("\n"))
ds

variants = [
    (2, "MATH217"),
    (1, "COMPSCI300"),
    (11, "ECE570"),
    (12, "COMPSCI539"),
    (12, "COMPSCI540"),
    (12, "GENBUS656"),
    (12, "ISYE521"),
    (12, "MATH535"),
    (12, "COMPSCI539"),
    (8, "COMPSCI412"),
    (8, "COMPSCI471"),
    (8, "COMPSCI513"),
    (8, "COMPSCI576"),
    (9, "ECON410"),
    (9, "STAT309"),
    (10, "COMPSCI525"),
    (10, "COMPSCI533"),
]
 
ds_g = sequence_graph.VariantGraph(ds)
for var in variants:
    ds_g.add_substitution_variant(*var)
ds_g.reorder_topological()
ds_g.plot()
ds_g.is_in_topological_order()

0 COMPSCI220
1 MATH221
2 MATH222
3 STAT240
4 COMPSCI320
5 MATH320
6 STAT340
7 COMPSCI400
8 ECON400
9 COMPSCI425
10 LIS461
11 COMPSCI532


True

<svg class="toyplot-canvas-Canvas" xmlns:toyplot="http://www.sandia.gov/toyplot" xmlns:xlink="http://www.w3.org/1999/xlink" xmlns="http://www.w3.org/2000/svg" width="1140.0px" height="660.0px" viewBox="0 0 1140.0 660.0" preserveAspectRatio="xMidYMid meet" style="background-color:transparent;border-color:#292724;border-style:none;border-width:1.0;fill:rgb(16.1%,15.3%,14.1%);fill-opacity:1.0;font-family:Helvetica;font-size:12px;opacity:1.0;stroke:rgb(16.1%,15.3%,14.1%);stroke-opacity:1.0;stroke-width:1.0" id="t097b793847ec4bfea37de48fb78808da"> COMPSCI300 COMPSCI220 MATH217 MATH221 MATH222 STAT240 COMPSCI320 MATH320 STAT340 COMPSCI576 COMPSCI513 COMPSCI471 COMPSCI412 COMPSCI400 STAT309 ECON410 ECON400 COMPSCI533 COMPSCI525 COMPSCI425 ECE570 LIS461 COMPSCI539 MATH535 ISYE521 GENBUS656 COMPSCI540 COMPSCI539 COMPSCI532

In [270]:
ds_g.num_vertices()

31

In [271]:
gg = """CHEM103CHEM104
MATH221
STAT371
CHEM341
PHYSICS103PHYSICS104
BIOLOGY151BIOLOGY152
BIOCHEM501
GENETICS467GENETICS468
GENETICS545
GENETICS520
GENETICS522
BIOCHEM508""".split("\n")

l=sorted(gg, key=lambda item: int(re.findall(r"\d{3}", item)[0]))
[(l[i], i+1) for i in range(len(l))]

gg = "|".join(sorted(gg, key=lambda item: int(re.findall(r"\d{3}", item)[0])))
gg

'CHEM103CHEM104|PHYSICS103PHYSICS104|BIOLOGY151BIOLOGY152|MATH221|CHEM341|STAT371|GENETICS467GENETICS468|BIOCHEM501|BIOCHEM508|GENETICS520|GENETICS522|GENETICS545'

In [272]:
variants = [
    (4, "MATH171|MATH217"),
    (6, "STAT301"),
    (6, "STAT240"),
    (1, "CHEM109"),
    (1, "CHEM115|CHEM116"),
    (5, "CHEM343CHEM345"),
    (2, "PHYSICS201|PHYSICS202"),
    (2, "PHYSICS207|PHYSICS208"),
    (3, "BOTANY130|ZOOLOGY101|ZOOLOGY102"),
    (3, "BIOCORE381|BIOCORE383"),
    (8, "BIOCHEM507"),
    (7, "GENETICS466"),
    (12, "GENETICS299"),
    (10, "GENETICS525"),
    (11, "GENETICS527"),
    (10, "GENETICS528"),
    (9, "BIOCHEM560"),
    (9, "COMPSCI576"),
    (9, "BIOCHEM570")
]
 
gg_g = sequence_graph.VariantGraph(gg)
for var in variants:
    gg_g.add_substitution_variant(*var)
    
gg_g.reorder_topological()
gg_g.plot()

0 CHEM103CHEM104
1 PHYSICS103PHYSICS104
2 BIOLOGY151BIOLOGY152
3 MATH221
4 CHEM341
5 STAT371
6 GENETICS467GENETICS468
7 BIOCHEM501
8 BIOCHEM508
9 GENETICS520
10 GENETICS522
11 GENETICS545


<svg class="toyplot-canvas-Canvas" xmlns:toyplot="http://www.sandia.gov/toyplot" xmlns:xlink="http://www.w3.org/1999/xlink" xmlns="http://www.w3.org/2000/svg" width="1380.0px" height="420.0px" viewBox="0 0 1380.0 420.0" preserveAspectRatio="xMidYMid meet" style="background-color:transparent;border-color:#292724;border-style:none;border-width:1.0;fill:rgb(16.1%,15.3%,14.1%);fill-opacity:1.0;font-family:Helvetica;font-size:12px;opacity:1.0;stroke:rgb(16.1%,15.3%,14.1%);stroke-opacity:1.0;stroke-width:1.0" id="t6a952fb7ca1c4a6bbc564c0c44f0cc6f"> CHEM115 CHEM116 CHEM109 CHEM103CHEM104 PHYSICS207 PHYSICS208 PHYSICS201 PHYSICS202 PHYSICS103PHYSICS104 BIOCORE381 BIOCORE383 BOTANY130 ZOOLOGY101 ZOOLOGY102 BIOLOGY151BIOLOGY152 MATH171 MATH217 MATH221 CHEM343CHEM345 CHEM341 STAT240 STAT301 STAT371 GENETICS466 GENETICS467GENETICS468 BIOCHEM507 BIOCHEM501 BIOCHEM570 COMPSCI576 BIOCHEM560 BIOCHEM508 GENETICS528 GENETICS525 GENETICS520 GENETICS527 GENETICS522 GENETICS299 GENETICS545

In [273]:
ds

'COMPSCI220|MATH221|MATH222|STAT240|COMPSCI320|MATH320|STAT340|COMPSCI400|ECON400|COMPSCI425|LIS461|COMPSCI532'

In [274]:
# diamond_g = sequence_graph.VariantGraph()
# diamond_g.add_substitution_variant(1, "COMPSCI300")
# diamond_g.reorder_topological()
# diamond_g.plot()

In [275]:
linear_g = sequence_graph.VariantGraph("COMPSCI220|ECE204|STAT240")
linear_g.reorder_topological()
linear_g.plot()

0 COMPSCI220
1 ECE204
2 STAT240


<svg class="toyplot-canvas-Canvas" xmlns:toyplot="http://www.sandia.gov/toyplot" xmlns:xlink="http://www.w3.org/1999/xlink" xmlns="http://www.w3.org/2000/svg" width="420.0px" height="180.0px" viewBox="0 0 420.0 180.0" preserveAspectRatio="xMidYMid meet" style="background-color:transparent;border-color:#292724;border-style:none;border-width:1.0;fill:rgb(16.1%,15.3%,14.1%);fill-opacity:1.0;font-family:Helvetica;font-size:12px;opacity:1.0;stroke:rgb(16.1%,15.3%,14.1%);stroke-opacity:1.0;stroke-width:1.0" id="t30dd424184454745b1c1ce242e9b4635"> COMPSCI220 ECE204 STAT240

In [276]:
ds_g._vertex_labels
gg_g._vertex_labels

[None,
 'CHEM115',
 'CHEM116',
 'CHEM109',
 'CHEM103CHEM104',
 'PHYSICS207',
 'PHYSICS208',
 'PHYSICS201',
 'PHYSICS202',
 'PHYSICS103PHYSICS104',
 'BIOCORE381',
 'BIOCORE383',
 'BOTANY130',
 'ZOOLOGY101',
 'ZOOLOGY102',
 'BIOLOGY151BIOLOGY152',
 'MATH171',
 'MATH217',
 'MATH221',
 'CHEM343CHEM345',
 'CHEM341',
 'STAT240',
 'STAT301',
 'STAT371',
 'GENETICS466',
 'GENETICS467GENETICS468',
 'BIOCHEM507',
 'BIOCHEM501',
 'BIOCHEM570',
 'COMPSCI576',
 'BIOCHEM560',
 'BIOCHEM508',
 'GENETICS528',
 'GENETICS525',
 'GENETICS520',
 'GENETICS527',
 'GENETICS522',
 'GENETICS299',
 'GENETICS545',
 None]

In [277]:
"|".join([item for item in ds_g._vertex_labels if item]) + "|" + "|".join([item for item in gg_g._vertex_labels if item])

'COMPSCI300|COMPSCI220|MATH217|MATH221|MATH222|STAT240|COMPSCI320|MATH320|STAT340|COMPSCI576|COMPSCI513|COMPSCI471|COMPSCI412|COMPSCI400|STAT309|ECON410|ECON400|COMPSCI533|COMPSCI525|COMPSCI425|ECE570|LIS461|COMPSCI539|MATH535|ISYE521|GENBUS656|COMPSCI540|COMPSCI539|COMPSCI532|CHEM115|CHEM116|CHEM109|CHEM103CHEM104|PHYSICS207|PHYSICS208|PHYSICS201|PHYSICS202|PHYSICS103PHYSICS104|BIOCORE381|BIOCORE383|BOTANY130|ZOOLOGY101|ZOOLOGY102|BIOLOGY151BIOLOGY152|MATH171|MATH217|MATH221|CHEM343CHEM345|CHEM341|STAT240|STAT301|STAT371|GENETICS466|GENETICS467GENETICS468|BIOCHEM507|BIOCHEM501|BIOCHEM570|COMPSCI576|BIOCHEM560|BIOCHEM508|GENETICS528|GENETICS525|GENETICS520|GENETICS527|GENETICS522|GENETICS299|GENETICS545'

In [278]:
align_graphs(ds_g, gg_g, basic_dna_submatrix, 0)

(3,
 [('COMPSCI300', ' '),
  ('COMPSCI220', '-'),
  (' ', 'CHEM115'),
  (' ', 'CHEM116'),
  (' ', 'CHEM109'),
  ('-', 'CHEM103CHEM104'),
  (' ', 'PHYSICS207'),
  (' ', 'PHYSICS208'),
  (' ', 'PHYSICS201'),
  (' ', 'PHYSICS202'),
  ('-', 'PHYSICS103PHYSICS104'),
  (' ', 'BIOCORE381'),
  (' ', 'BIOCORE383'),
  (' ', 'BOTANY130'),
  (' ', 'ZOOLOGY101'),
  (' ', 'ZOOLOGY102'),
  ('-', 'BIOLOGY151BIOLOGY152'),
  ('MATH217', ' '),
  (' ', 'MATH171'),
  (' ', 'MATH217'),
  ('MATH221', 'MATH221'),
  ('MATH222', '-'),
  (' ', 'CHEM343CHEM345'),
  ('-', 'CHEM341'),
  ('STAT240', 'STAT240'),
  ('COMPSCI320', '-'),
  ('MATH320', '-'),
  ('STAT340', '-'),
  (' ', 'STAT301'),
  (' ', 'STAT371'),
  (' ', 'GENETICS466'),
  ('-', 'GENETICS467GENETICS468'),
  (' ', 'BIOCHEM507'),
  ('-', 'BIOCHEM501'),
  (' ', 'BIOCHEM570'),
  ('COMPSCI576', 'COMPSCI576'),
  ('COMPSCI513', ' '),
  ('COMPSCI471', ' '),
  ('COMPSCI412', ' '),
  ('COMPSCI400', ' '),
  ('STAT309', ' '),
  ('ECON410', ' '),
  ('ECON400', '-'

In [279]:
self.num_vertices() >= 2 
                self.is_in_topological_order() and 
                self.is_labeled()

IndentationError: unexpected indent (2498753362.py, line 2)

In [280]:
ds_g.num_vertices()

31

In [266]:
ds_g.is_in_topological_order()

True

In [223]:
ds_g.is_labeled()

True